In [1]:
import sys 
print(sys.executable)

/Users/doga/Desktop/chatbot arena project/cbot/bin/python


In [2]:
import pandas as pd
import fastparquet
import pyarrow
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity


In [3]:
df = pd.read_parquet("hf://datasets/lmsys/chatbot_arena_conversations/data/train-00000-of-00001-cced8514c7ed782a.parquet")
df.head(5)

,question_id,model_a,model_b,winner,judge,conversation_a,conversation_b,turn,anony,language,tstamp,openai_moderation,toxic_chat_tag
0,58210e39b3fd4441a2bd4a518bb44c2d,chatglm-6b,koala-13b,model_b,arena_user_973,[{'content': 'What is the difference between O...,[{'content': 'What is the difference between O...,1,True,English,1.682352e+09,"{'categories': {'harassment': False, 'harassme...","{'roberta-large': {'flagged': False, 'probabil..."
1,2564acd09e3942fd97657d05282d4389,oasst-pythia-12b,alpaca-13b,tie,arena_user_973,[{'content': 'Why did my parent not invite me ...,[{'content': 'Why did my parent not invite me ...,1,True,English,1.682352e+09,"{'categories': {'harassment': False, 'harassme...","{'roberta-large': {'flagged': False, 'probabil..."
2,90bfd142157948aba01931726c888e7f,koala-13b,oasst-pythia-12b,model_b,arena_user_973,"[{'content': 'Fuji vs. Nikon, which is better?...","[{'content': 'Fuji vs. Nikon, which is better?...",1,True,English,1.682352e+09,"{'categories': {'harassment': False, 'harassme...","{'roberta-large': {'flagged': False, 'probabil..."
3,a7c5accc53e649a3bc6b2e41d962ebc4,vicuna-13b,oasst-pythia-12b,model_b,arena_user_973,[{'content': 'How to build an arena for chatbo...,[{'content': 'How to build an arena for chatbo...,1,True,English,1.682352e+09,"{'categories': {'harassment': False, 'harassme...","{'roberta-large': {'flagged': False, 'probabil..."
4,adf27e819a3c494cb6e993f0c660e097,vicuna-13b,koala-13b,model_a,arena_user_973,"[{'content': 'When is it today?', 'role': 'use...","[{'content': 'When is it today?', 'role': 'use...",1,True,English,1.682352e+09,"{'categories': {'harassment': False, 'harassme...","{'roberta-large': {'flagged': False, 'probabil..."


In [4]:
#{"role": "user", "content": "..."} -> [0]
#{"role": "assistant", "content": "..."} -> [1]
#content matches dictionay prompt

In [5]:
def extract_info(row):
    try:
        prompt = row["conversation_a"][0]["content"]
        response_a = row["conversation_a"][1]["content"]
        response_b = row["conversation_b"][1]["content"]
        winner = row["winner"]
        return pd.Series([prompt, response_a, response_b, winner])
    except Exception:
        return pd.Series([None, None, None, None])

df_extracted = df.apply(extract_info, axis=1)
df_extracted.columns = ["prompt", "response_a", "response_b", "human_winner"]
df_extracted.dropna(inplace=True)
# df_extracted.to_csv("chatbot_arena_extracted.csv", index=False)
df_extracted.head(5)

,prompt,response_a,response_b,human_winner
0,What is the difference between OpenCL and CUDA?,OpenCL and CUDA are two different programming ...,OpenCL and CUDA are both programming languages...,model_b
1,Why did my parent not invite me to their wedding?,It is possible that your parent did not invite...,It is likely that they wanted to keep the gues...,tie
2,"Fuji vs. Nikon, which is better?",Both Fuji and Nikon are popular camera brands ...,This is a subjective question and the answer d...,model_b
3,How to build an arena for chatbots?,Building an arena for chatbots can be done by ...,Building an arena for chatbots is a great way ...,model_b
4,When is it today?,"I'm sorry, I cannot determine the current date...","Today is February 23, 2023.",model_a


CLASSIFIER

In [6]:
#randomly sample 100 rows
sampled_df = df_extracted.sample(n=100, random_state=42).reset_index(drop=True)

In [12]:
# GPT scoring 
import openai
import time

openai.api_key = ["YOUR_OPENAI_API_KEY"]

def gpt4_judgment(prompt, response_a, response_b):
    system_prompt = """You are a helpful assistant that evaluates the factual correctness, coherence, and clarity of two AI-generated responses to a user prompt. You must pick which one is better, or state if they are equal."""
    
    user_prompt = f"""
Prompt: {prompt}

Response A: {response_a}

Response B: {response_b}

Which response is better? Reply with one of: 'A', 'B', or 'tie'.
"""

    try:
        res = openai.ChatCompletion.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            temperature=0
        )
        decision = res['choices'][0]['message']['content'].strip().lower()
        if 'a' in decision and 'b' not in decision:
            return "model_a"
        elif 'b' in decision and 'a' not in decision:
            return "model_b"
        elif "tie" in decision:
            return "tie"
        else:
            return "unknown"
    except Exception as e:
        print("Error:", e)
        return "error"


In [13]:
#row labeling
def label_row(row):
    gpt_winner = gpt4_judgment(row["prompt"], row["response_a"], row["response_b"])
    row["gpt_winner"] = gpt_winner
    row["label"] = int(gpt_winner == row["human_winner"]) if gpt_winner in ["model_a", "model_b"] else None
    return row

labeled_df = sampled_df.apply(label_row, axis=1)
#labeled_df.to_csv("labeled_chatbot_judgments.csv", index=False)
labeled_df.head(5)


Error: The model `gpt-4` does not exist or you do not have access to it.
Error: The model `gpt-4` does not exist or you do not have access to it.
Error: The model `gpt-4` does not exist or you do not have access to it.
Error: The model `gpt-4` does not exist or you do not have access to it.
Error: The model `gpt-4` does not exist or you do not have access to it.
Error: The model `gpt-4` does not exist or you do not have access to it.
Error: The model `gpt-4` does not exist or you do not have access to it.
Error: The model `gpt-4` does not exist or you do not have access to it.
Error: The model `gpt-4` does not exist or you do not have access to it.
Error: The model `gpt-4` does not exist or you do not have access to it.
Error: The model `gpt-4` does not exist or you do not have access to it.
Error: The model `gpt-4` does not exist or you do not have access to it.
Error: The model `gpt-4` does not exist or you do not have access to it.
Error: The model `gpt-4` does not exist or you do n

,prompt,response_a,response_b,human_winner,gpt_winner,label
0,write me a weeding speech in Albanian,"Here it is:\n\n""Albana, the land of the Albani...","Përshëndetje të gjithëve,\n\nSot jemi të bashk...",model_b,error,None
1,ISO 26262: technical requiremen for functional...,The technical requirements for the functional ...,ISO 26262 is a standard for the development of...,model_a,error,None
2,what is the Adam,Adam is a biblical figure in the Hebrew Bible ...,Adam is a first-order optimization algorithm t...,model_a,error,None
3,Schreibe ein Gedicht über Kaninchen.,"In einem Land, so grün und weit,\nda hüpft ein...","Zarte Ohren, flauschiges Fell,\nKaninchen hopp...",tie,error,None
4,explique moi comment je peux ressembler a lair...,L'art de ressembler à Laird Miehud est un exer...,dans le futur vous pouvez vous présenter comme...,tie (bothbad),error,None


In [9]:
model = SentenceTransformer('all-MiniLM-L6-v2')

def encode_text(text):
    return model.encode(text)

#calculate similarity between two embeddings
def calculate_similarity(embedding_a, embedding_b):
    return cosine_similarity([embedding_a], [embedding_b])[0][0]

In [ ]:
#encode the text
labeled_df['prompt_embedding'] = labeled_df['prompt'].apply(encode_text)
labeled_df['response_a_embedding'] = labeled_df['response_a'].apply(encode_text)
labeled_df['response_b_embedding'] = labeled_df['response_b'].apply(encode_text)

#calculate similarity between the embeddings
labeled_df['similarity_a_b'] = labeled_df.apply(lambda row: calculate_similarity(row['prompt_embedding'], row['response_a_embedding']), axis=1)
labeled_df['similarity_a_c'] = labeled_df.apply(lambda row: calculate_similarity(row['prompt_embedding'], row['response_b_embedding']), axis=1)
labeled_df['similarity_b_c'] = labeled_df.apply(lambda row: calculate_similarity(row['response_a_embedding'], row['response_b_embedding']), axis=1)

labeled_df.head(5)


In [11]:
def combine_features(row):
    return np.concatenate([
        row['prompt_embedding'],
        row['response_a_embedding'],
        row['response_b_embedding'],
        [row['similarity_a_b']],
        [row['similarity_a_c']],
        [row['similarity_b_c']]
    ])

labeled_df['features'] = labeled_df.apply(combine_features, axis=1)
labeled_df.head(5)


,prompt,response_a,response_b,human_winner,gpt_winner,label,prompt_embedding,response_a_embedding,response_b_embedding,similarity_a_b,similarity_a_c,similarity_b_c,features
0,write me a weeding speech in Albanian,"Here it is:\n\n""Albana, the land of the Albani...","Përshëndetje të gjithëve,\n\nSot jemi të bashk...",model_b,error,None,"[0.00090263964, 0.08146245, -0.019265478, 0.02...","[-0.019966735, 0.118588835, -0.0032393907, 0.0...","[-0.05448748, 0.08408137, 0.09397893, -0.03848...",0.561416,0.231599,0.093901,"[0.0009026396437548101, 0.08146245032548904, -..."
1,ISO 26262: technical requiremen for functional...,The technical requirements for the functional ...,ISO 26262 is a standard for the development of...,model_a,error,None,"[-0.050106883, 0.01170723, -0.077458605, 0.010...","[-0.032367595, 0.018893026, -0.064028434, 0.00...","[-0.01591786, 0.010488646, -0.079126835, 0.027...",0.769802,0.766539,0.711753,"[-0.05010688304901123, 0.011707229539752007, -..."
2,what is the Adam,Adam is a biblical figure in the Hebrew Bible ...,Adam is a first-order optimization algorithm t...,model_a,error,None,"[-0.031983335, 0.029641649, 0.0231351, -0.0011...","[0.035256937, 0.06237943, 0.00058803515, 0.027...","[-0.09135917, -0.022418719, 0.03546467, -0.037...",0.716471,0.525721,0.343163,"[-0.03198333457112312, 0.0296416487544775, 0.0..."
3,Schreibe ein Gedicht über Kaninchen.,"In einem Land, so grün und weit,\nda hüpft ein...","Zarte Ohren, flauschiges Fell,\nKaninchen hopp...",tie,error,None,"[0.092044756, 0.090163745, 0.012006649, 0.0057...","[-0.03834474, 0.017420044, -0.037667282, 0.019...","[-0.0320743, 0.013008292, -0.031750686, 0.0277...",0.348308,0.356203,0.593323,"[0.09204475581645966, 0.09016374498605728, 0.0..."
4,explique moi comment je peux ressembler a lair...,L'art de ressembler à Laird Miehud est un exer...,dans le futur vous pouvez vous présenter comme...,tie (bothbad),error,None,"[-0.062433362, 0.039559808, 0.0035601135, -0.0...","[-0.0604297, 0.014287303, -0.0125871375, -0.12...","[-0.042159673, 0.05592871, -0.056610413, -0.03...",0.490012,0.308307,0.352538,"[-0.06243336200714111, 0.03955980762839317, 0...."


In [12]:
# #prepare labels
# labeled_df = labeled_df.dropna(subset=['label'])
# labels = labeled_df['label'].values

# #split
# from sklearn.model_selection import train_test_split

# X = np.stack(labeled_df['features'].values)
# y = labels

# #train a random forest classifier
# from sklearn.ensemble import RandomForestClassifier

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# clf = RandomForestClassifier()
# clf.fit(X_train, y_train)

# print("Accuracy:", clf.score(X_test, y_test))

In [13]:
#filter 0
misaligned_df = labeled_df[labeled_df["label"] == 0].copy()
misaligned_df.head(5)
misaligned_df.shape[0]


0

In [16]:
if len(misaligned_df) == 0:
    print("No misaligned samples available!")
else:
    sampled_data = misaligned_df.sample(n=500, random_state=42, replace=True)
    print(sampled_data.head(5))


No misaligned samples available!


In [17]:
import spacy
from spacy.matcher import Matcher

In [18]:
nlp = spacy.load("en_core_web_sm")


In [19]:
#classify closed ended and open ended prompts
patterns = [
    {"label": "MATH", "pattern": [{"LOWER": {"IN": [
        "add", "subtract", "multiply", "divide", "equation", "integral", "derivative", "mathematics",
        "algebra", "geometry", "trigonometry", "calculus", "limit", "logarithm", "exponent", "modulus",
        "matrix", "vector", "scalar", "mean", "median", "mode", "standard deviation", "variance",
        "probability", "distribution", "combination", "permutation", "function", "identity", "root",
        "square", "cube", "angle", "theorem", "proof", "hypothesis", "corollary", "lemma"
    ]}}]},
    
    {"label": "NUMBER", "pattern": [{"LIKE_NUM": True}]},
    
    {"label": "CODE", "pattern": [{"LOWER": {"IN": [
        "function", "variable", "loop", "class", "object", "array", "list", "dictionary",
        "string", "integer", "float", "boolean", "if", "else", "elif", "switch", "case", "for",
        "while", "break", "continue", "return", "import", "def", "try", "except", "finally",
        "with", "as", "yield", "lambda", "none", "true", "false"
    ]}}]}
]


In [20]:
matcher = Matcher(nlp.vocab)
for pattern in patterns:
    matcher.add(pattern["label"], [pattern["pattern"]])


In [21]:
def detect_terms(prompt):
    doc = nlp(prompt)
    matches = matcher(doc)
    detected_terms = set()
    for match_id, start, end in matches:
        span = doc[start:end]
        detected_terms.add((nlp.vocab.strings[match_id], span.text))
    return detected_terms

In [ ]:
sampled_data['detected_terms'] = sampled_data['prompt'].apply(detect_terms)
sampled_data.head(5)

NameError: name 'sampled_data' is not defined

In [21]:
def classify_closed_ended(detected_terms):
    # If any terms are detected, classify as closed-ended
    if detected_terms:
        return 'Closed-ended'
    else:
        return 'Open-ended'

sampled_data['category'] = sampled_data['detected_terms'].apply(classify_closed_ended)
sampled_data.head(5)

,prompt,response_a,response_b,human_winner,gpt_winner,label,prompt_embedding,response_a_embedding,response_b_embedding,similarity_a_b,similarity_a_c,similarity_b_c,features,detected_terms,category
98,Tell me a joke,Why did the carrot turn red?\n\nBecause it saw...,A construction worker was walking when crashed...,model_b,model_a,0.0,"[-0.013610002, -0.035938635, 0.02055572, 0.002...","[-0.050809518, -0.025566049, -0.0257295, 0.102...","[-0.05158351, 0.022370216, 0.03135999, 0.06952...",0.156050,0.210762,0.170802,"[-0.013610001653432846, -0.03593863546848297, ...",{},Open-ended
64,What is the industrial revolution?,The Industrial Revolution refers to a period o...,The industrial revolution was a significant pe...,model_b,model_a,0.0,"[-0.025486542, 0.0061054714, -0.045459997, -0....","[-0.0524989, -0.004233326, 0.00977435, 0.00209...","[-0.012855443, 0.004006175, 0.021647064, -0.02...",0.718032,0.749566,0.858142,"[-0.025486541911959648, 0.0061054714024066925,...",{},Open-ended
28,what is a Calabi-Yau manifold ?\n\nAnswer:,A Calabi-Yau manifold is a type of mathematica...,A Calabi-Yau manifold is a complex manifold th...,model_b,model_a,0.0,"[-0.010430147, 0.028473515, -0.055231307, -0.0...","[-0.03338751, -0.04724372, -0.050808728, 0.015...","[-0.047239825, 0.028733192, -0.03291552, 0.004...",0.734846,0.858537,0.844449,"[-0.01043014694005251, 0.028473515063524246, -...",{},Open-ended
18,"""Write a C++ program to find the nth Fibonacci...",#include<iostream>\n\nusing namespace std;\n\n...,Here is a C++ program to find the nth Fibonacc...,model_b,model_a,0.0,"[-0.036859207, 0.10353846, -0.018434316, -0.02...","[-0.009891856, 0.08116202, 0.014044625, -0.064...","[-0.09364377, 0.040925734, 0.016252827, -0.078...",0.709220,0.824649,0.755761,"[-0.03685920685529709, 0.10353846102952957, -0...",{},Open-ended
39,"Given\na=2, b=3, c=6;\na=1.4, b=5, c=7;\na=2, ...","In order to solve this problem, we need to fin...",We need to find the value of c given the value...,tie (bothbad),model_a,0.0,"[-0.003208244, 0.044472877, -0.047849033, 0.01...","[0.019388301, 0.074543595, -0.018501896, -0.03...","[0.00016731117, 0.049990736, -0.048009012, -0....",0.611023,0.702953,0.777603,"[-0.0032082439865916967, 0.04447287693619728, ...","{(CODE, If)}",Closed-ended


In [ ]:
selected_columns = ['prompt', 'response_a', 'response_b', 'human_winner', 'gpt_winner', 'label', 'category']
filtered_data = sampled_data[selected_columns]
filtered_data.head(500)



NameError: name 'sampled_data' is not defined

In [ ]:
from save_to_csv import save_dataframe_to_csv

save_dataframe_to_csv(filtered_data, filename='filtered_data', output_dir='/Users/doga/Desktop/chatbot arena project')
print(f"\n✅ Saved disguised outputs")

In [23]:
category_counts = filtered_data['category'].value_counts()
total = category_counts.sum()
percentages = (category_counts / total) * 100

print("Percentage of Open-ended and Closed-ended Prompts:")
print(percentages)





Percentage of Open-ended and Closed-ended Prompts:
category
Open-ended      55.6
Closed-ended    44.4
Name: count, dtype: float64
